In [30]:
import os
import sys

path = os.getcwd() + "/data/mesivta/"
file = path + "train.txt"

In [31]:
!apt install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.



before downloading and unpacking the KenLM repo.

In [32]:
import platform
import os

if platform.system() == 'Darwin':  # Darwin stands for MacOS
    !curl -L https://kheafield.com/code/kenlm.tar.gz | tar xz
elif platform.system() == 'Linux':
    !wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz
else:
    print("Unsupported operating system")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

100  480k  100  480k    0     0   382k      0  0:00:01  0:00:01 --:--:--  382k


KenLM is written in C++, so we'll make use of `cmake` to build the binaries.

In [42]:
!mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2
!ls kenlm/build/bin

To reinstall 3.28.3, run:
  brew reinstall cmake
==> Downloading https://ghcr.io/v2/homebrew/core/boost/manifests/1.84.0_1
######################################################################### 100.0%
==> Fetching dependencies for boost: icu4c
==> Downloading https://ghcr.io/v2/homebrew/core/icu4c/manifests/74.2
######################################################################### 100.0%
==> Fetching icu4c
==> Downloading https://ghcr.io/v2/homebrew/core/icu4c/blobs/sha256:3facc22a7821
######################################################################### 100.0%
==> Fetching boost
==> Downloading https://ghcr.io/v2/homebrew/core/boost/blobs/sha256:5249ac98f677
######################################################################### 100.0%
==> Installing dependencies for boost: icu4c
==> Installing boost dependency: icu4c
==> Downloading https://ghcr.io/v2/homebrew/core/icu4c/manifests/74.2
Already downloaded: /Users/shua/Library/Caches/Homebrew/downloads/3e5230e726e5548db28b

In [34]:
!pip install pyctcdecode
!pip install kenlm -U

Using cached pyctcdecode-0.5.0-py2.py3-none-any.whl (39 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.3/449.3 kB 2.6 MB/s eta 0:00:00a 0:00:01
Using cached pygtrie-2.5.0-py3-none-any.whl (25 kB)
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
  Using cached kenlm-0.2.0-cp311-cp311-macosx_14_0_arm64.whl

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


Great, as we can see, the executable functions have successfully been built under `kenlm/build/bin/`.

KenLM by default computes an *n-gram* with [Kneser-Ney smooting](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing). All text data used to create the *n-gram* is expected to be stored in a text file.
We download our dataset and save it as a `.txt` file.

Now, we just have to run KenLM's `lmplz` command to build our *n-gram*, called `"5gram.arpa"`. As it's relatively common in speech recognition, we build a *5-gram* by passing the `-o 5` parameter.
For more information on the different *n-gram* LM that can be built
with KenLM, one can take a look at the [official website of KenLM](https://kheafield.com/code/kenlm/).

Executing the command below might take a minute or so.

In [35]:

!kenlm/build/bin/lmplz -o 5 <"{file}" > "{path}/5gram.arpa"

zsh:1: no such file or directory: kenlm/build/bin/lmplz


Great, we have built a *5-gram* LM! Let's inspect the first couple of lines.

In [36]:
!head -20 5gram.arpa

head: 5gram.arpa: No such file or directory


There is a small problem that 🤗 Transformers will not be happy about later on.
The *5-gram* correctly includes a "Unknown" or `<unk>`, as well as a *begin-of-sentence*, `<s>` token, but no *end-of-sentence*, `</s>` token.
This sadly has to be corrected currently after the build.

We can simply add the *end-of-sentence* token by adding the line `0 </s>  -0.11831701` below the *begin-of-sentence* token and increasing the `ngram 1` count by 1. Because the file has roughly 100 million lines, this command will take *ca.* 2 minutes.

In [37]:
with open(f"{path}5gram.arpa", "r") as read_file, open(f"{path}5gram_correct.arpa", "w") as write_file:
  has_added_eos = False
  for line in read_file:
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)

Let's now inspect the corrected *5-gram*.

In [38]:
!head -20 {path}5gram_correct.arpa

Great, this looks better! We're done at this point and all that is left to do is to correctly integrate the `"ngram"` with [`pyctcdecode`](https://github.com/kensho-technologies/pyctcdecode) and 🤗 Transformers.

## Create the Processor

In [39]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("imvladikon/wav2vec2-xls-r-300m-hebrew")

/Users/shua/Desktop/FinalProject/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


Next, we extract the vocabulary of its tokenizer as it represents the `"labels"` of `pyctcdecode`'s `BeamSearchDecoder` class.

In [40]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

The `"labels"` and the previously built `5gram_correct.arpa` file is all that's needed to build the decoder.

In [41]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path="5gram_correct.arpa",
)

OSError: Cannot read model '5gram_correct.arpa' (util/file.cc:76 in int util::OpenReadOrThrow(const char *) threw ErrnoException because `-1 == (ret = open(name, 0x0000))'. No such file or directory while opening /Users/shua/Desktop/FinalProject/word2vec-kenlm/5gram_correct.arpa)

We can safely ignore the warning and all that is left to do now is to wrap the just created `decoder`, together with the processor's `tokenizer` and `feature_extractor` into a `Wav2Vec2ProcessorWithLM` class.

In [ ]:
from transformers import Wav2Vec2ProcessorWithLM

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

We want to directly upload the LM-boosted processor into
the model folder of [`xls-r-300m-sv`](https://huggingface.co/hf-test/xls-r-300m-sv) to have all relevant files in one place.

Let's clone the repo, add the new decoder files and upload them afterward.
First, we need to install `git-lfs`.

In [ ]:
# !sudo apt-get install git-lfs tree

Cloning and uploading of modeling files can be done conveniently with the `huggingface_hub`'s `Repository` class.

More information on how to use the `huggingface_hub` to upload any files, please take a look at the [official docs](https://huggingface.co/docs/hub/how-to-upstream).

In [ ]:
# from huggingface_hub import Repository

# repo = Repository(local_dir="xls-r-300m-sv", clone_from="hf-test/xls-r-300m-sv")

Having cloned `xls-r-300m-sv`, let's save the new processor with LM into it.

In [ ]:
# processor_with_lm.save_pretrained("xls-r-300m-sv")

Let's inspect the local repository. The `tree` command conveniently can also show the size of the different files.

In [ ]:
# !tree -h xls-r-300m-sv/

As can be seen the *5-gram* LM is quite large - it amounts to more than 4 GB.
To reduce the size of the *n-gram* and make loading faster, `kenLM` allows converting `.arpa` files to binary ones using the `build_binary` executable.

Let's make use of it here.

In [ ]:
# !kenlm/build/bin/build_binary xls-r-300m-sv/language_model/5gram_correct.arpa xls-r-300m-sv/language_model/5gram.bin

Great, it worked! Let's remove the `.arpa` file and check the size of the binary *5-gram* LM.

In [ ]:
# !rm xls-r-300m-sv/language_model/5gram_correct.arpa && tree -h xls-r-300m-sv/

Nice, we reduced the *n-gram* by more than half to less than 2GB now. In the final step, let's upload all files.

In [ ]:
# repo.push_to_hub(commit_message="Upload lm-boosted decoder")

In [ ]:
#save the processor to local
processor_with_lm.save_pretrained("/teamspace/studios/this_studio/FinalProject/models/KenLM-Wav2Vec2-Hebrew")